In [7]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

# conectar base de datos
conn=sql.connect('Data/movies2.db')
cur=conn.cursor()

# ver tablas disponibles en base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('f_ratings',)]

# **2. Sistemas de recomendación contenido general**
Si un usuario ve películas de géneros múltiples (por ejemplo, acción y comedia), el sistema sugiere películas que sean híbridas entre esos géneros para mantener su interés.<br>
Si un usuario ve películas de géneros variados y de épocas específicas, el sistema sugiere películas que combinen esos géneros y sean de épocas similares. Las recomendaciones se actualizan semanalmente, ofreciendo 10 películas alineadas con sus gustos en géneros y épocas.<br>
Este sistema se enfoca en sugerir películas híbridas entre los géneros que el usuario ha visto previamente, pero también considera la época de las películas (basado en su año de lanzamiento). Si el usuario ha visto películas de acción y comedia, el sistema recomendará películas que combinen estos géneros y que pertenezcan a épocas cercanas a las que el usuario prefiere, por ejemplo, si muestra interés en películas de los 80s o 90s, las recomendaciones respetarán ese contexto temporal. El sistema actualiza las recomendaciones semanalmente, revisando el historial de visualización del usuario para detectar patrones no solo en los géneros, sino también en las épocas.<br> 
Cada semana, se generan 10 recomendaciones frescas que incluyen películas que combinan géneros y épocas que el usuario ha disfrutado previamente, brindando una experiencia más inmersiva y personalizada.

In [8]:
df = pd.read_sql("SELECT * FROM f_ratings", conn)
df.head()

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,fecha_nueva,movie_year,clean_title
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000-07-30,1995,Toy Story
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,2000-07-30,1995,Grumpier Old Men
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,2000-07-30,1995,Heat
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2000-07-30,1995,Seven (a.k.a. Se7en)
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2000-07-30,1995,"Usual Suspects, The"


In [9]:
# géneros
unique_genres = set()
for genres in df['movie_genres'].str.split('|'):
    unique_genres.update(genres)

# Contar la cantidad de géneros únicos
num_unique_genres = len(unique_genres)
print("Número de géneros diferentes:", num_unique_genres)

# Imprimir los géneros diferentes
print("Géneros diferentes:")
for genre in unique_genres:
    print(genre)

Número de géneros diferentes: 19
Géneros diferentes:
Western
Adventure
IMAX
Musical
Drama
Horror
War
Animation
Film-Noir
Thriller
Fantasy
Romance
Children
Mystery
Comedy
Crime
Action
Sci-Fi
Documentary


In [10]:
# Ajustar la configuración de visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

pregunta. vamos a poner todos en un solo notebook?, de lo contrario hacer la copia no sobraría?

In [11]:
df2 = df.copy()
# Convertir la variable de género en una lista de géneros para cada película
df2['movie_genres_list'] = df2['movie_genres'].str.split('|')

# Obtener todas las categorías únicas de géneros
unique_genres = set()
for genres_list in df2['movie_genres_list']:
    unique_genres.update(genres_list)

# Convertir la lista de géneros en variables dummy y agregarlas al DataFrame original
for genre in unique_genres:
    df2[genre] = df2['movie_genres_list'].apply(lambda x: 1 if genre in x else 0)

# Eliminar la columna temporal 'movie_genres_list'
df2.drop(columns=['movie_genres_list'], inplace=True)

# Mostrar las primeras filas del DataFrame resultante
df2.head()

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,fecha_nueva,movie_year,clean_title,Western,Adventure,IMAX,Musical,Drama,Horror,War,Animation,Film-Noir,Thriller,Fantasy,Romance,Children,Mystery,Comedy,Crime,Action,Sci-Fi,Documentary
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000-07-30,1995,Toy Story,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,2000-07-30,1995,Grumpier Old Men,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,2000-07-30,1995,Heat,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2000-07-30,1995,Seven (a.k.a. Se7en),0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2000-07-30,1995,"Usual Suspects, The",0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0


In [13]:
# Eliminar las columnas del DataFrame
peliculas = df2.drop(columns=['user_id', 'movie_id', 'rating', 'movie_title', 'movie_genres', 'fecha_nueva']).copy()
peliculas.reset_index(drop=True, inplace=True)
peliculas['movie_year'] = peliculas['movie_year'].fillna(0).astype(int)
# Eliminar duplicados basados en la columna 'movie_clean_title'
peliculas = peliculas.drop_duplicates(subset=['clean_title'])
# escalar el año de publicacion
sc = MinMaxScaler()
peliculas.loc[:, "movie_year"] = sc.fit_transform(peliculas[['movie_year']])
# Reiniciar el índice
peliculas.reset_index(drop=True, inplace=True)

peliculas

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_9444\462067698.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.73417722 0.73417722 0.73417722 0.73417722 0.73417722 0.74683544
 0.73417722 0.73417722 0.73417722 0.73417722 0.72151899 0.72151899
 0.72151899 0.50632911 0.72151899 0.72151899 0.73417722 0.72151899
 0.72151899 0.72151899 0.72151899 0.70886076 0.70886076 0.70886076
 0.70886076 0.70886076 0.70886076 0.70886076 0.70886076 0.67088608
 0.65822785 0.6835443  0.03797468 0.74683544 0.74683544 0.74683544
 0.74683544 0.74683544 0.74683544 0.74683544 0.02531646 0.05063291
 0.05063291 0.17721519 0.74683544 0.74683544 0.43037975 0.53164557
 0.69620253 0.62025316 0.69620253 0.56962025 0.65822785 0.48101266
 0.5443038  0.63291139 0.55696203 0.43037975 0.53164557 0.58227848
 0.67088608 0.53164557 0.29113924 0.5443038  0.63291139 0.59493671
 0.5443038  0.70886076 0.60759494 0.62025316 0.46835443 0.03797468
 0.65822785

,timestamp,movie_year,clean_title,Western,Adventure,IMAX,Musical,Drama,Horror,War,Animation,Film-Noir,Thriller,Fantasy,Romance,Children,Mystery,Comedy,Crime,Action,Sci-Fi,Documentary
0,964982703,0.734177,Toy Story,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
1,964981247,0.734177,Grumpier Old Men,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,964982224,0.734177,Heat,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0
3,964983815,0.734177,Seven (a.k.a. Se7en),0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,964982931,0.734177,"Usual Suspects, The",0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,1113190465,0.835443,Underworld,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
854,1355184567,0.658228,Cinema Paradiso (Nuovo cinema Paradiso),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
855,1464275372,0.848101,Harold and Kumar Go to White Castle,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
856,1464197299,0.873418,"Devil Wears Prada, The",0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
# dummies
dummies = peliculas.drop(columns=['clean_title'])
dummies

,timestamp,movie_year,Western,Adventure,IMAX,Musical,Drama,Horror,War,Animation,Film-Noir,Thriller,Fantasy,Romance,Children,Mystery,Comedy,Crime,Action,Sci-Fi,Documentary
0,964982703,0.734177,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
1,964981247,0.734177,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
2,964982224,0.734177,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0
3,964983815,0.734177,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,964982931,0.734177,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,1113190465,0.835443,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
854,1355184567,0.658228,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
855,1464275372,0.848101,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
856,1464197299,0.873418,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [15]:
##### ### entrenar modelo #####
## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar)
model = neighbors.NearestNeighbors(n_neighbors = 6, metric='cosine')
model.fit(dummies)
dist, idlist = model.kneighbors(dummies)
distancias = pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila (pelicula)
id_list = pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde


In [17]:
def MovieRecommender(movie_name=list(peliculas['clean_title'].value_counts().index)):
    pelicula_list_name = []
    pelicula_id = peliculas[peliculas['clean_title'] == movie_name].index
    pelicula_id = pelicula_id[0]
    # Obtener los vecinos más cercanos excluyendo la propia película
    neighbor_ids = [newid for newid in idlist[pelicula_id] if newid != pelicula_id]
    # Obtener los nombres de las películas vecinas
    for newid in neighbor_ids:
        pelicula_list_name.append(peliculas.loc[newid].movie_clean_title)
    recomendaciones = pd.DataFrame(pelicula_list_name, columns=['Peliculas Recomendadas'])
    return recomendaciones

print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Toy Story', "Howl's Moving Castle (Hauru no…

<function MovieRecommender at 0x000001DB074CB7E0>
